In [8]:
import os
from pathlib import Path
import splitfolders
from zipfile import ZipFile

In [9]:
os.chdir("..")

In [10]:
Path.cwd()

WindowsPath('c:/Users/USER/Desktop/Farm-Insects-Classification')

In [11]:
DATA_PATH = Path("archive.zip").resolve()

In [12]:
DATA_PATH

WindowsPath('C:/Users/USER/Desktop/Farm-Insects-Classification/archive.zip')

In [13]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    unzip_dir: Path

In [14]:
from FarmInsectsClassifier.constants import CONFIG_FILE_PATH, PARAM_FILE_PATH
from FarmInsectsClassifier.utils import create_directories, read_yaml
from FarmInsectsClassifier.logger import logging

In [16]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 param_file_path = PARAM_FILE_PATH):
        
        
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAM_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion


        logging.info("creating artifacts directory")
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = Path(config.root_dir),
            unzip_dir = Path(config.unzip_dir)

        )

        return data_ingestion_config

In [17]:
class DataIngestion:
    def __init__(self, data_path: Path, config: DataIngestionConfig):
        self.data_path = data_path
        self.config = config

    def unzip(self) -> None:
        unzip_path = self.config.unzip_dir
        unzip_path.mkdir(exist_ok=True, parents=True)

        logging.info("Extracting zip file")

        with ZipFile(self.data_path, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

        logging.info("Zipfile extraction completed")


    def split_data(self) ->  None:
        path = list(self.config.unzip_dir.resolve().iterdir())[0]
        output = self.config.unzip_dir / "farm-insects-splitted"

        logging.info("Splitting folder into train, test and validation set")

        splitfolders.ratio(path, seed=1, output=str(output), ratio=(0.6, 0.2, 0.2))

        logging.info("Train, test and validation data successfully created")

            

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_path=DATA_PATH, config=data_ingestion_config)
    data_ingestion.unzip()
    data_ingestion.split_data()

except Exception as e:
    raise e

Copying files: 0 files [00:00, ? files/s]
